In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

selected_columns = selected_columns[['Client', 'Client ID', 'Query ID', 'Query', 'Class', 'Record Type', 'Flags', 'IP Address', 'domain']]

selected_columns['Query ID'] = pd.to_numeric(selected_columns['Query ID'], errors='coerce')
selected_columns = selected_columns.dropna(subset=['Query ID'])

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(selected_columns['Query ID'].values.reshape(-1, 1))

def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60

X, y = create_dataset(scaled_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, batch_size=32, epochs=10)

train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

plt.figure(figsize=(14,7))
plt.plot(range(len(y_train_actual)), y_train_actual, color='blue', label='Training data')
plt.plot(range(len(y_train_actual), len(y_train_actual) + len(y_test_actual)), y_test_actual, color='green', label='Testing data')
plt.plot(range(len(y_train_actual)), train_predict, color='red', label='Training prediction')
plt.plot(range(len(y_train_actual), len(y_train_actual) + len(y_test_actual)), test_predict, color='orange', label='Testing prediction')
plt.title('Query ID Prediction')
plt.xlabel('Index')
plt.ylabel('Query ID')
plt.legend()
plt.show()

